In [ ]:
!unzip MULTI_CLASS_SEGM_DEMO.zip


In [1]:
from edge import set_project_name, set_workspace
set_project_name("deeplab_demo")
set_workspace("/tmp")

from edge.segmentation.deeplab_v3.evaluation import CocoEvaluator

from edge.segmentation.deeplab_v3 import train
from edge.metrics.cocostuff_modules.cocostuffeval import COCOStuffeval
from edge.segmentation.deeplab_v3 import get_inference_model
from edge.mlflowlite_logger import get_experiment_id

from mlflow_lite import set_experiment
from mlflow_lite import log_metrics
set_experiment('521e7ebc-36c9-4ae8-8507-f7b31a5bd963_19')

import tensorflow as tf

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval


from logzero import logger

In [2]:
train_images = 'MULTI_CLASS_SEGM_DEMO/TRAIN/IMAGES'
train_annotations = 'MULTI_CLASS_SEGM_DEMO/TRAIN/MASK'

eval_images = 'MULTI_CLASS_SEGM_DEMO/TEST'

# analytics phase

In [5]:
from edge.segmentation import get_analytics
from pprint import pprint

analytics_result = get_analytics(
        image_dir = train_images,
        annotation_dir = train_annotations
)

pprint(analytics_result)

100%|██████████| 500/500 [00:05<00:00, 88.37it/s]

Total images for training: 500
{'black_images': 0,
 'label_mappings': {'1': 1,
                    '10': 9,
                    '11': 10,
                    '12': 11,
                    '13': 12,
                    '14': 13,
                    '15': 14,
                    '2': 2,
                    '255': 15,
                    '3': 3,
                    '4': 4,
                    '5': 5,
                    '6': 6,
                    '7': 7,
                    '8': 8},
 'negative_images': 0,
 'num_classes': 15,
 'positive_images': 500,
 'total': 500}


# prepare data

In [6]:
from edge.segmentation import prepare_data
from edge.metrics import convert_pascalvoc_coco

prepare_data()

convert_pascalvoc_coco(
    image_dir=train_images,
    annotation_dir=train_annotations,
    coco_gt_path='/tmp/segm_gt.json',
    iouType='segm')

100%|██████████| 500/500 [02:37<00:00,  3.17it/s]


# train

In [3]:
for i in [5, 10]:
    train(
          resize_height=600,
          resize_width=600,
          num_epochs=i,
          batch_size=2,
          checkpoint_prefix='test_deeplab',
          snapshot_every_epoch=5,
          output_stride=16,
          atrous_rates=(6,12,18),
          checkpoint_path=None,
          steps_per_epoch=None,
          initial_epoch=i-5,
          weights='pascal_voc', # pascal_voc, cityscapes, path to h5 file trained with edge
          aug=None, # coming soon
          lr=1e-5,
          backbone_network='xception', # xception, mobilenetv2
          print_summary=False,
          mlflowlite_log_step=200,
          distribute_strategy=None)


    logger.info('getting inference model ..')
    checkpoint_file = '/tmp/deeplab_demo/test_deeplab_{}.h5'.format(i)
    pred_model = get_inference_model(checkpoint_file)

    logger.info('creating coco style predictions ..')
    json_saver = CocoEvaluator(
      model=pred_model, 
      coco_gt='/tmp/segm_gt.json', #get this from above
      coco_dt='/tmp/segm_pred.json') 
    #json_saver.run(image_dir=None, threshold=0.5)

    cocoGt = COCO('/tmp/segm_gt.json')
    cocoDt = cocoGt.loadRes('/tmp/segm_pred.json')

    #logger.info('Evaluating metrics on COCO-Stuff-Evaluation ...')
    #CocoStuffEvaluator = COCOStuffeval(cocoGt,cocoDt,0,255)
    #CocoStuffEvaluator.evaluate()
    #stats, statsClass = CocoStuffEvaluator.summarize()

    logger.info('Evaluating metrics on COCO Evaluation ...')
    cocoEval = COCOeval(cocoGt=cocoGt, cocoDt=cocoDt, iouType='segm')
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()

    if get_experiment_id():
          logger.info('logging to mlflow-lite')
          log_metrics(metrics=statsClass, step=num_epoch)
          log_metrics(
                metrics={
                'miou':stats[0].item(),
                'fwiou':stats[1].item(),
                'mean-acc':stats[2].item(),
                'pixel-acc':stats[3].item()},
                step=num_epoch)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


[I 200426 12:00:20 <ipython-input-3-fecc709d05cb>:20] Logging to mlflow-lite under exp-id 521e7ebc-36c9-4ae8-8507-f7b31a5bd963_19


[I 200426 12:00:20 <ipython-input-3-fecc709d05cb>:20] Logging to mlflow-lite under exp-id 521e7ebc-36c9-4ae8-8507-f7b31a5bd963_19
2020/04/26 12:00:21 WARNING mlflow_lite.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet



logged training data to server
Train on 500 steps
Epoch 1/5
199/500 [==========>...................] - ETA: 2:41 - loss: 10.6121 - categorical_accuracy: 0.2612 - precision: 0.7515 - recall: 0.1717

[I 200426 12:02:26 callbacks:236] Logged 'batch' metrics to mlflow-lite
[I 200426 12:02:26 callbacks:236] Logged 'size' metrics to mlflow-lite
[I 200426 12:02:27 callbacks:236] Logged 'loss' metrics to mlflow-lite
[I 200426 12:02:27 callbacks:236] Logged 'categorical_accuracy' metrics to mlflow-lite
[I 200426 12:02:27 callbacks:236] Logged 'precision' metrics to mlflow-lite
[I 200426 12:02:27 callbacks:236] Logged 'recall' metrics to mlflow-lite


...





399/500 [======================>.......] - ETA: 44s - loss: 1.4305 - categorical_accuracy: 0.7559 - precision: 0.7708 - recall: 0.8986

[I 200426 12:18:59 callbacks:236] Logged 'batch' metrics to mlflow-lite
[I 200426 12:18:59 callbacks:236] Logged 'size' metrics to mlflow-lite
[I 200426 12:18:59 callbacks:236] Logged 'loss' metrics to mlflow-lite
[I 200426 12:18:59 callbacks:236] Logged 'categorical_accuracy' metrics to mlflow-lite
[I 200426 12:18:59 callbacks:236] Logged 'precision' metrics to mlflow-lite
[I 200426 12:18:59 callbacks:236] Logged 'recall' metrics to mlflow-lite






499/500 [============================>.] - ETA: 0s - loss: 1.4421 - categorical_accuracy: 0.7561 - precision: 0.7707 - recall: 0.8978
Epoch 00005: saving model to /tmp/deeplab_demo/checkpoints/test_deeplab_05.h5
500/500 [==============================] - 336s 672ms/step - loss: 1.4421 - categorical_accuracy: 0.7562 - precision: 0.7709 - recall: 0.8978


[I 200426 12:21:37 <ipython-input-3-fecc709d05cb>:23] getting inference model ..


OSError: Unable to open file (unable to open file: name = '/tmp/deeplab_demo/test_deeplab_5.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)